In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss,roc_auc_score
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings("ignore")

Test_data = pd.read_csv('/content/drive/MyDrive/Applied AI/CS1/Final/Test_Final.csv')

X_Test = Test_data.drop(['Y'], axis=1)
y_Test = Test_data['Y'].values

print('X_Test:',X_Test.shape,'\ny_Test:',y_Test.shape)

X_Test: (2030, 25) 
y_Test: (2030,)


## 1.1 Final function that computes prediction of a single datapoint

In [ ]:
def final_function_1(X): # X is a single datapoint

  X = X.drop(['car','direction_opp','toCoupon_GEQ5min'], axis=1)

  # mode imputation for missing values in data point
  frequent_df = pd.read_csv('/content/drive/MyDrive/Applied AI/CS1/Final/frequent_values.csv',index_col=0)
  for i in (X.columns.values):
    if (X[i].isnull().values.any()):
      X[i] = X[i].fillna(frequent_df.loc[i][0])

  # FE-1
  if (list(X['toCoupon_GEQ15min'])[0] == 0):
    X['to_Coupon'] = 'within15min'
  elif (list(X['toCoupon_GEQ15min'])[0] == 1)and(list(X['toCoupon_GEQ25min'])[0] == 0):
    X['to_Coupon'] = 'within25min'
  else:
    X['to_Coupon'] = 'morethan25min'
  # FE-2
  if (list(X['coupon'])[0] == 'Restaurant(<20)'):
        X['coupon_freq'] = (list(X['RestaurantLessThan20'])[0])
  elif (list(X['coupon'])[0] == 'Coffee House'):
        X['coupon_freq'] = (list(X['CoffeeHouse'])[0])
  elif (list(X['coupon'])[0] == 'Carry out & Take away'):
        X['coupon_freq'] = (list(X['CarryAway'])[0])
  elif (list(X['coupon'])[0] == 'Bar'):
        X['coupon_freq'] = (list(X['Bar'])[0])
  elif (list(X['coupon'])[0] == 'Restaurant(20-50)'):
        X['coupon_freq'] = (list(X['Restaurant20To50'])[0])
  # FE-3
  occupation_dict = {'Healthcare Support':'High_Acceptance','Construction & Extraction':'High_Acceptance','Healthcare Practitioners & Technical':'High_Acceptance',
                   'Protective Service':'High_Acceptance','Architecture & Engineering':'High_Acceptance','Production Occupations':'Medium_High_Acceptance',
                    'Student':'Medium_High_Acceptance','Office & Administrative Support':'Medium_High_Acceptance','Transportation & Material Moving':'Medium_High_Acceptance',
                    'Building & Grounds Cleaning & Maintenance':'Medium_High_Acceptance','Management':'Medium_Acceptance','Food Preparation & Serving Related':'Medium_Acceptance',
                   'Life Physical Social Science':'Medium_Acceptance','Business & Financial':'Medium_Acceptance','Computer & Mathematical':'Medium_Acceptance',
                    'Sales & Related':'Medium_Low_Acceptance','Personal Care & Service':'Medium_Low_Acceptance','Unemployed':'Medium_Low_Acceptance',
                   'Farming Fishing & Forestry':'Medium_Low_Acceptance','Installation Maintenance & Repair':'Medium_Low_Acceptance','Education&Training&Library':'Low_Acceptance',
                    'Arts Design Entertainment Sports & Media':'Low_Acceptance','Community & Social Services':'Low_Acceptance','Legal':'Low_Acceptance','Retired':'Low_Acceptance'}
  X['occupation_class'] = X['occupation'].map(occupation_dict)
  X = X.drop(['occupation'], axis=1)

  # Encoding
  vectorizer = pickle.load(open("/content/drive/MyDrive/Applied AI/CS1/Final/vectorizer.pkl", "rb"))
  X_Ordinal_encoding = vectorizer.transform(X)
  X_Ordinal_encoding = pd.DataFrame(X_Ordinal_encoding,columns=(X.columns.values)+'_OrE')

  # load trained model
  model = pickle.load(open("/content/drive/MyDrive/Applied AI/CS1/Final/model.pkl", "rb"))
  y_pred = model.predict(X_Ordinal_encoding)
  y_pred_prob = model.predict_proba(X_Ordinal_encoding)

  return y_pred[0], y_pred_prob[0]

In [ ]:
datapoint = X_Test.iloc[[212]]
Predicted_output, Predicted_prob = final_function_1(datapoint)
print('Actual output for given datapoint is',y_Test[212])
print('Predicted output for given datapoint is',Predicted_output)
print('Predicted probability for given datapoint is',Predicted_prob)

Actual output for given datapoint is 0
Predicted output for given datapoint is 0
Predicted probability for given datapoint is [0.88290001 0.11709999]


In [ ]:
datapoint = X_Test.iloc[[356]]
Predicted_output, Predicted_prob = final_function_1(datapoint)
print('Actual output for given datapoint is',y_Test[356])
print('Predicted output for given datapoint is',Predicted_output)
print('Predicted probability for given datapoint is',Predicted_prob)

Actual output for given datapoint is 1
Predicted output for given datapoint is 1
Predicted probability for given datapoint is [0.11612079 0.88387921]


## 1.2 Final function that computes metric of the unseen dataset

In [ ]:
def final_function_2(X_set,y_set): # X_set & y_set is an unseen dataset
  Y_predict = []
  Y_predict_prob = []
  for i in tqdm(range(X_set.shape[0])):
    df = pd.DataFrame([(X_set.iloc[i]).values],columns=(X_set.iloc[i]).index)
    y_pred, y_pred_prob = final_function_1(df)
    Y_predict.append(y_pred)
    Y_predict_prob.append(y_pred_prob)
    
  logloss = log_loss(y_set,np.array(Y_predict_prob))
  AUC_Score = roc_auc_score(y_set,np.array(Y_predict_prob)[:,1])
  
  return logloss, AUC_Score

logloss, AUC_Score  = final_function_2(X_Test,y_Test)
print('\nTest log loss:',logloss,'\nTest AUC Score:',AUC_Score)

100%|██████████| 2030/2030 [2:41:18<00:00,  4.77s/it]


Test log loss: 0.4884936156148184 
Test AUC Score: 0.8435276742400861
